In [1]:
import boto3
import sagemaker
from sagemaker import Model
import os
import json

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_session = sagemaker.Session()

role = sagemaker.get_execution_role()
print(f"SageMaker Role ARN: {role}")

SageMaker Role ARN: arn:aws:iam::796932308591:role/service-role/SageMaker-ExecutionRole-20250214T145019


## Normalization

In [3]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline

In [4]:
s3 = boto3.client('s3')

bucket = 'mitrailabs-personaclassification'
model_prefix = 'risk_prediction/data/'
response = s3.get_object(
    Bucket=bucket,
    Key=f"{model_prefix}feature_engineered_data.csv"
)

X_processed = pd.read_csv(response['Body'])
X_processed_copy = X_processed.copy()

In [5]:
numerical_cols = X_processed.columns
# numerical_pipeline = Pipeline([
#     ('scaler', MinMaxScaler())
# ])
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_pipeline, numerical_cols),
#         # ('cat', categorical_pipeline, categorical_cols)
#     ]
# )


# X_scaled = preprocessor.fit_transform(X_processed)
# X_processed = pd.DataFrame(
#     X_scaled, 
#     columns=numerical_cols#+categorical_cols#[name.split('__')[-1] for name in preprocessor.get_feature_names_out()]
# )

In [6]:
scaler = MinMaxScaler()  # Default range is (0,1)
X_scaled = scaler.fit_transform(X_processed) 
# X_processed

In [7]:
# inference_preprocessor = ColumnTransformer(
#     transformers=[
#         # ('num', numerical_pipeline, numerical_cols),
#         ('cat', numerical_pipeline, [i for i in numerical_cols if i != "target_default"])
#     ],
#     # remainder='drop'  # <--- IMPORTANT: Drop the remaining columns
# )

# inference_preprocessor.fit(X_processed_copy.drop(columns=["target_default"]))

# import joblib

# # For the second save (which appears duplicative but keeping structure)
# inference_preprocessor_path = 'saved/inference_preprocessor.joblib'
# joblib.dump(inference_preprocessor, inference_preprocessor_path)

# # Upload to S3 (updating file extension)
# target_bucket = 'mitrailabs-personaclassification'
# target_prefix = 'risk_prediction/Intermediate_states'
# model_path = f'{target_prefix}/inference_preprocessor.joblib'
# s3.upload_file(
#     inference_preprocessor_path,
#     target_bucket,
#     model_path
# )

In [8]:
# !pip freeze > requirements2.txt


## Clustering

In [9]:
from sklearn.cluster import KMeans

# Perform K-Means clustering on normalized data
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels to the original dataset
X_processed['Cluster'] = clusters

print(pd.Series(clusters).value_counts().sort_index())
# Display the clustered data
(X_processed.head())






X_processed_path = 'saved/clustered_data.csv'
X_processed.to_csv(X_processed_path, index=False)

target_bucket = 'mitrailabs-personaclassification'
target_prefix = 'risk_prediction/data'


model_path = f'{target_prefix}/clustered_data.csv'
s3.upload_file(X_processed_path, 
               target_bucket, 
               model_path
              )

0    10696
1    10457
2     9223
3     3478
4     7887
Name: count, dtype: int64


In [12]:
X_processed.columns

Index(['target_default', 'score_1', 'score_2', 'score_3', 'score_4', 'score_5',
       'score_6', 'risk_rate', 'last_amount_borrowed',
       'last_borrowed_in_months', 'credit_limit', 'income', 'facebook_profile',
       'state', 'real_state', 'ok_since', 'n_bankruptcies',
       'n_defaulted_loans', 'n_accounts', 'n_issues',
       'application_time_in_funnel',
       'external_data_provider_credit_checks_last_month',
       'external_data_provider_credit_checks_last_year',
       'external_data_provider_email_seen_before',
       'external_data_provider_fraud_score', 'reported_income',
       'shipping_state', 'score_4_minus_score_3', 'avg_score_5_6',
       'debt_to_income', 'credit_utilization', 'default_rate',
       'reported_income_div_income', 'fraud_score_bin', 'credit_available',
       'income_per_account', 'loan_amount_to_income',
       'n_accounts_to_credit_limit', 'debt_to_income_x_default_rate',
       'credit_utilization_x_fraud_score', 'income_sq',
       'last_amoun